In [3]:
import numpy as np
import os
os.chdir('Shaique/ProbR/GBM/Adhoc')
os.getcwd()
import pandas as pd
from datetime import datetime
import numpy as np

In [4]:
df=pd.read_csv("listings.csv")

/opt/python/python35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13913,https://www.airbnb.com/rooms/13913,20190710134553,2019-07-11,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,...,f,f,moderate,f,f,3,1,2,0,0.15


# Data Exploration and Feature engineering 

In [10]:
df.market.value_counts()

London                         82977
Other (International)             10
Plymouth                           4
Other (Domestic)                   2
Paris                              2
Herefordshire and Mid Wales        1
Oahu                               1
Lancashire                         1
Emilia Romagna Countryside         1
Reims                              1
Leeds                              1
Orlando                            1
Suffolk                            1
La Baule and Noirmoutier           1
Name: market, dtype: int64

In [ ]:
#we see that market is always london

In [28]:
city_list=["Greater London","London",
"Twickenham",
"Richmond",
"Croydon",
"Londres",
"Ilford",
"Kingston upon Thames",
"Harrow",
"Wembley"]
df.loc[(df.city=="Greater London"),'city_numeric']=1
df.loc[(df.city=="London"),'city_numeric']=2
df.loc[(df.city=="Twickenham"),'city_numeric']=3
df.loc[(df.city=="Richmond"),'city_numeric']=4
df.loc[(df.city=="Croydon"),'city_numeric']=5
df.loc[(df.city=="Londres"),'city_numeric']=6

df.loc[(df.city=="Ilford"),'city_numeric']=7
df.loc[(df.city=="Kingston upon Thames"),'city_numeric']=8
df.loc[(df.city=="Harrow"),'city_numeric']=9
df.loc[(df.city=="Wembley"),'city_numeric']=10
df.loc[(~df.city.isin(city_list)),'city_numeric']=11




In [ ]:
#converted into numbers so that it can be fed t the model as well

In [25]:
df.city_numeric.value_counts()

1.0     44228
2.0     33977
11.0     4448
3.0       181
4.0       180
5.0       165
6.0       158
7.0       130
8.0       130
9.0       127
10.0      126
Name: city_numeric, dtype: int64

In [39]:
property_list=["Apartment",
"House",
"Townhouse",
"Serviced apartment",
"Condominium",
"Bed and breakfast"]
df.loc[(df.property_type=="Apartment"),'property_numeric']=1
df.loc[(df.property_type=="House"),'property_numeric']=2
df.loc[(df.property_type=="Townhouse"),'property_numeric']=3
df.loc[(df.property_type=="Serviced apartment"),'property_numeric']=4
df.loc[(df.property_type=="Condominium"),'property_numeric']=5
df.loc[(df.property_type=="Bed and breakfast"),'property_numeric']=6
df.loc[(~df.property_type.isin(property_list)),'property_numeric']=7



In [40]:
df.property_numeric.value_counts()

1.0    56802
2.0    17436
3.0     2979
7.0     2753
4.0     1896
5.0     1107
6.0      877
Name: property_numeric, dtype: int64

In [41]:
df.room_type.value_counts()

Entire home/apt    47142
Private room       35972
Shared room          736
Name: room_type, dtype: int64

In [ ]:
#price has $ attched to it and also .00 at the the end which we need to remove 

In [60]:
df['price_2'] = df['price'].astype(str).str[1:-3].astype(str)
df['price_2']= df['price_2'].astype(str).apply(lambda x: x.replace(',', ''))
df['price_2'] = df['price_2'].astype(np.int32)

In [64]:
df['price_2'].head()

0     65
1    100
2    300
3    175
4     65
Name: price_2, dtype: int32

In [70]:
df['price_2'].describe()

count    83850.000000
mean       121.656303
std        224.900151
min          0.000000
25%         45.000000
50%         81.000000
75%        140.000000
max      13700.000000
Name: price_2, dtype: float64

In [72]:
ps = []

for x in df['price_2']:
    if x <=45:
        ps.append("1")
    elif x <=81:
        ps.append("2")
    elif x <=140:
        ps.append("3")
    elif x <=200:
        ps.append("4")
    elif x>200:
        ps.append("5")

df["price_band"] = ps

In [73]:
df["price_band"].value_counts()

3    21844
1    21151
2    20831
4    10418
5     9606
Name: price_band, dtype: int64

In [65]:
df['price_2'].quantile(.12)

34.0

In [69]:
df['price_2'].describe()

count    83850.000000
mean       121.656303
std        224.900151
min          0.000000
25%         45.000000
50%         81.000000
75%        140.000000
max      13700.000000
Name: price_2, dtype: float64

In [ ]:
# we have many prices like weekly and monthly prices which can be converted into numerci form by the use of following function 

In [95]:
def dollar_converted(x):
    df[x].fillna(0,inplace=True)
    df['x2'] = df[x].astype(str).str[1:-3].astype(str)
    df['x2']= df['x2'].astype(str).apply(lambda x: x.replace(',', ''))
    df['price_2']= df['price_2'].astype(str).apply(lambda x: x.replace('', '999'))
    df['x2'] = df['x2'].astype(np.int32)
    return(df['x2'])
    

In [104]:
df["review_scores_rating"].value_counts()

100.0    17642
98.0      3906
93.0      3691
96.0      3638
97.0      3598
95.0      3362
90.0      3226
80.0      3219
99.0      2453
94.0      2293
92.0      1811
91.0      1651
87.0      1472
88.0      1288
89.0      1254
85.0       845
60.0       773
86.0       686
84.0       670
83.0       612
70.0       357
82.0       324
73.0       301
20.0       272
81.0       207
40.0       202
75.0       181
78.0       176
76.0       170
77.0       145
67.0       123
79.0       122
50.0        78
74.0        71
72.0        56
65.0        55
68.0        35
71.0        34
64.0        27
53.0        27
69.0        24
55.0        16
63.0        13
47.0        13
66.0         9
62.0         8
30.0         7
57.0         4
56.0         2
52.0         2
49.0         2
61.0         2
51.0         1
35.0         1
33.0         1
45.0         1
44.0         1
46.0         1
Name: review_scores_rating, dtype: int64

In [102]:
df["cancellation_policy"].value_counts()

strict_14_with_grace_period    35878
flexible                       27126
moderate                       19957
super_strict_60                  469
super_strict_30                  391
luxury_moderate                   17
strict                             9
luxury_super_strict_95             3
Name: cancellation_policy, dtype: int64

In [ ]:
df.to_csv("listings2.csv")

(13850, 112)

# modelling with XGBOOST

In [ ]:
import time
start = time.time()
import pickle

import os
os.chdir('Shaique/ProbR/GBM/Adhoc/')
os.getcwd()
import pandas as pd

#splitting inti train and test 
df = pd.read_csv("listings2.csv")

df[:7000].to_csv("listings_train.csv")
df[70000:83850].to_csv("listings_test.csv")


fields =["city_numeric","review_scores_rating","property_numeric","price_2",,"minimum_nights","maximum_nights",
         "minimum_minimum_nights","maximum_minimum_nights","availability_30","availability_60","cubhiseg",
         "availability_90","availability_365","number_of_reviews","review_scores_rating","review_scores_accuracy","review_scores_cleanliness"
        "review_scores_checkin","review_scores_communication","review_scores_location","review_scores_value","calculated_host_listings_count",
        "calculated_host_listings_count_entire_homes","calculated_host_listings_count_private_rooms","calculated_host_listings_count_shared_rooms",
         "reviews_per_month","accommodates","bathrooms","bedrooms","beds"]

file = pd.read_csv("listings_train.csv",usecols=fields)


file.drop(fields,axis=1,inplace=True)

print(file.columns)



file.fillna(-9999, inplace=True)

len(fields)

y = pd.DataFrame(file["price_2"])

h=pd.DataFrame(file.drop('price_2', axis=1))


import xgboost as xgb
print(xgb.__version__)

dtrain = xgb.DMatrix(h, label=y)

param = {
    'max_depth': 5,  
    'learning_rate': .1,  
    'objective':'reg:linear',
    'min_child_weight':100,
    'num_class' : 5,
    'tree_method':'exact'
    
}

print(file.columns)


model = xgb.train(param, dtrain,num_boost_round=1500)

filename = 'model_xgb_Listings.sav'
pickle.dump(model, open(filename, 'wb'))

print(file.columns)
end = time.time()
print(end - start)


# scoring The XGBOOST

In [ ]:
fields =["city_numeric","review_scores_rating","property_numeric","price_2",,"minimum_nights","maximum_nights",
         "minimum_minimum_nights","maximum_minimum_nights","availability_30","availability_60","cubhiseg",
         "availability_90","availability_365","number_of_reviews","review_scores_rating","review_scores_accuracy","review_scores_cleanliness"
        "review_scores_checkin","review_scores_communication","review_scores_location","review_scores_value","calculated_host_listings_count",
        "calculated_host_listings_count_entire_homes","calculated_host_listings_count_private_rooms","calculated_host_listings_count_shared_rooms",
         "reviews_per_month","accommodates","bathrooms","bedrooms","beds"]

file = pd.read_csv("listings_test.csv",usecols=fields)
print(file.columns)

file.fillna(file.median(), inplace=True)

y = pd.DataFrame(file["price_2"])

h=pd.DataFrame(file.drop('price_2', axis=1))


import xgboost as xgb
print(xgb.__version__)

dtest = xgb.DMatrix(h, label=y)

filename = 'model_xgb_Listings.sav'
loaded_model = pickle.load(open(filename, 'rb'))

y_pred = loaded_model.predict(dtest,pred_leaf=False)

a = list(y_pred)


t=pd.DataFrame(y_pred,columns=["pred"])
t1 =round(t,2) 
t1["Actual_price"] = y

t1.to_csv("Listings_score.csv",index=False)

end = time.time()
print(end - start)